# JAMA revision round 4

In [0]:
!pip install lifelines

## Import packages



In [0]:
import pandas as pd, numpy as np, re
import gc
from numpy import exp, mean
from fancyimpute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from lifelines import CoxPHFitter, AalenJohansenFitter, KaplanMeierFitter
import warnings
warnings.filterwarnings('ignore')

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## **Read data**

In [0]:
# whole cohort (N=1,478,506)
# new cohort (N=1,537,928)
data_param = "Fortable3mi_ld_dec_censored2.csv" #@param ["Fortable3mi_ld_dec_censored.csv", "Fortable3mi_ld_dec_censored1.csv", "Fortable3mi_ld_dec_censored2.csv"]
d = pd.read_csv('drive/My Drive/facility/'+data_param) 
d['rucc_rural'] = np.where(d.RUCC_2013.isin([1, 2, 3]), 0, (np.where(d.RUCC_2013.isna(), np.nan, 1)))
d = d.drop(['RUCC_2013'], axis=1)

In [5]:
d.shape

(1587949, 52)

In [6]:
d.isna().sum()

USRDS_ID                              0
PROVUSRD                              0
dialysis_mod1                      7944
INC_AGE                               0
FIRST_SE                              0
sex_new                               0
bmi_35                                0
insurance_esrd                   121191
chf                                   0
ashd_new                              0
copd_new                              0
pvasc_new                             0
cva_new                               0
hypertension                          0
diabetes                              0
smoke_new                             0
cancer_new                            0
race_new                              0
age_cat                               0
other_cardiac                         0
esrd_cause                            0
PATTXOP_MEDUNFITn                     0
Mortality_Rate_Facility           15797
Hospitalization_Rate_facility     13085
smr_dfr                           10222


#**Multiple imputation**


##**Random forest imputation for categorical**

In [0]:
for var in ['dialysis_mod1', 'insurance_esrd', 'rucc_rural', 'nephcare_cat2']:
    pred = ['sex_new', 'age_cat', 'race_new', var]
    imputer = IterativeImputer(n_iter=1, random_state=7, predictor=RandomForestClassifier(n_estimators=10))
    imputed = pd.DataFrame(imputer.fit_transform(d[pred]), columns=pred)
    d = d.drop(var, axis=1).join(imputed[var])


##**Bayesian Ridge linear imputation for continuous**


In [0]:
for var in ['Hospitalization_Rate_facility', 'Mortality_Rate_Facility', 'NEAR_DIST']:
    completed = []
    for i in range(5):
        pred = ['sex_new', 'age_cat', 'race_new', var]
        imputer = IterativeImputer(n_iter=5, sample_posterior=True, random_state=i, min_value = d[var].min(), max_value=d[var].max())
        completed.append(imputer.fit_transform(d[pred]))
    completed_mean = np.mean(completed, axis=0)
    imputed = pd.DataFrame(completed_mean, columns=pred)
    # if var == 'NEAR_DIST':
    #     m = imputed[imputed.NEAR_DIST > 0].NEAR_DIST.mean()
    #     imputed.NEAR_DIST = np.where(imputed.NEAR_DIST < 0, m, imputed.NEAR_DIST)
    d = d.drop(var, axis=1).join(imputed[var])

#**Create cohort for Cox model**

**1) dummy code and order levels based on table**

**2) drop unneeded variables**

In [0]:
# standard cohort
PH_data = d[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2',  'profit_txc', 'profit_hosp', 'change_status',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data = PH_data.join(pd.get_dummies(PH_data.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.age_cat, [5, 1, 2, 3, 4, 6], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data = PH_data.join(pd.get_dummies(PH_data.race_new, prefix='race_new', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(pd.Categorical(PH_data.chain_class2, [6, 5, 2, 1, 3, 4], True), prefix='chain_class2', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_txc, prefix='profit_txc', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.profit_hosp, prefix='profit_hosp', drop_first=True))
PH_data = PH_data.join(pd.get_dummies(PH_data.change_status, prefix='change_status', drop_first=True))
PH_data = PH_data.drop(['dialysis_mod1',
            'esrd_cause', 
            'age_cat', 
            'race_new', 
            # 'chain_class2', 
            'change_status',
            'insurance_esrd', 
            'network_us_region_dfr', 
            'profit_txc', 
            'profit_hosp'
            ], axis=1)

# ideal cohort
ideal = d[(d.INC_AGE < 66) &
  (d.pvasc_new == 0) &
  (d.chf == 0) &
  (d.cva_new == 0) &
  (d.PATTXOP_MEDUNFITn == 0)].reset_index(drop=True)

PH_data_ideal = ideal[['PROVUSRD', 'chain_class2', 'for_profit', 'sex_new', 'age_cat', 'race_new', 'dialysis_mod1', 'esrd_cause', 'bmi_35',
                 'ashd_new', 'chf',	'other_cardiac', 'cva_new',	'pvasc_new', 'hypertension', 'diabetes', 'copd_new',
                 'smoke_new', 'cancer_new', 'insurance_esrd', 'PATTXOP_MEDUNFITn','nephcare_cat2',
                 'network_us_region_dfr', 'NEAR_DIST', 'rucc_rural', 'wl', 'wl_time', 'livingd', 'ld_time', 'deceasedt', 'dec_time']]
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.dialysis_mod1, prefix='dialysis_mod1', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.insurance_esrd, [3, 2, 1, 4, 5], True), prefix='insurance_esrd', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.age_cat, [5, 1, 2, 3, 4], True), prefix='age_cat', drop_first=True)) # delete category "6" for ideal cohort!
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.race_new, prefix='race_new', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.esrd_cause, prefix='esrd_cause', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(PH_data_ideal.network_us_region_dfr, prefix='network_us_region_dfr', drop_first=True))
PH_data_ideal = PH_data_ideal.join(pd.get_dummies(pd.Categorical(PH_data_ideal.chain_class2, [6,5,2,1,3,4], True), prefix='chain_class2', drop_first=True))
PH_data_ideal = PH_data_ideal.drop(['dialysis_mod1', 'esrd_cause', 'age_cat', 'race_new', 'chain_class2', 'insurance_esrd', 'network_us_region_dfr'], axis=1)

##Table 1

In [0]:
varlist=['chain_class2_',
         'for_profit',
          'age_cat',
         'sex_new',
         'race_new',
         'insurance_esrd',
         'esrd_cause',
         'dialysis_mod1',
         'bmi_35',
         'chf',
         'ashd_new',
         'other_cardiac',
         'cva_new',
         'pvasc_new',
         'hypertension',
         'diabetes',
         'copd_new',
         'smoke_new',
         'cancer_new',
         'nephcare_cat2',
         'PATTXOP_MEDUNFITn',
         'network_us_region_dfr',
         'rucc_rural',
         'NEAR_DIST']



##**Table 2**

In [12]:
# Truncate follow-up time
def truncate(t):
    e = PH_data.copy()
    if t>0:
      e.loc[e['wl_time'] > t, 'wl']=0
      e.loc[e['wl_time'] > t, 'wl_time']=t
      e.loc[e['ld_time'] > t, 'livingd']=0
      e.loc[e['ld_time'] > t, 'ld_time']=t
      e.loc[e['dec_time'] > t, 'deceasedt']=0
      e.loc[e['dec_time'] > t, 'dec_time']=t
    return e
  

cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in varlist:
    crude = '|'.join([exposure, time, status])
    cph.fit(truncate(0).filter(regex=crude), duration_col=time, event_col=status, step_size=0.5)
    print(round(pd.concat([cph.hazard_ratios_.rename('HR'), exp(cph.confidence_intervals_)], 1), 2))

------------------------------ wl ------------------------------
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.04             1.01             1.06
chain_class2_2  0.90             0.88             0.91
chain_class2_1  0.85             0.84             0.87
chain_class2_3  0.87             0.86             0.89
chain_class2_4  0.82             0.80             0.84
              HR  95% lower-bound  95% upper-bound
for_profit  0.86             0.85             0.87
             HR  95% lower-bound  95% upper-bound
age_cat_1  3.52             3.46             3.58
age_cat_2  2.59             2.55             2.62
age_cat_3  2.11             2.09             2.14
age_cat_4  1.63             1.61             1.65
age_cat_6  0.14             0.14             0.14
           HR  95% lower-bound  95% upper-bound
sex_new  0.74             0.74             0.75
              HR  95% lower-bound  95% upper-bound
race_new_2  1.26             1.25             1.27
race_

In [0]:
for case, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  tmp = d
  count = tmp.groupby('chain_class2')[case].sum().map('{:,}'.format)
  pyr = (tmp.groupby('chain_class2')[time].sum()/12).map(' /{:,.0f}'.format)
  print('-'*30, case,'-'*30)
  print(count+pyr)
  count1 = tmp.groupby('for_profit')[case].sum().map('{:,}'.format)
  pyr1 = (tmp.groupby('for_profit')[time].sum()/12).map(' /{:,.0f}'.format)
  print(count1+pyr1)

------------------------------ wl ------------------------------
chain_class2
1    69,559 /1,410,648
2    71,699 /1,350,990
3      33,398 /632,491
4      12,477 /239,514
5      13,827 /243,858
6      19,125 /338,530
dtype: object
for_profit
0       32,952 /582,388
1    187,133 /3,633,643
dtype: object
------------------------------ livingd ------------------------------
chain_class2
1    12,162 /1,562,201
2    11,867 /1,514,997
3       5,009 /712,671
4       2,034 /267,717
5       2,586 /275,982
6       3,374 /379,991
dtype: object
for_profit
0       5,960 /655,973
1    31,072 /4,057,586
dtype: object
------------------------------ deceasedt ------------------------------
chain_class2
1    27,031 /1,562,201
2    26,465 /1,514,997
3      12,330 /712,671
4       4,232 /267,717
5       5,218 /275,982
6       7,910 /379,991
dtype: object
for_profit
0      13,128 /655,973
1    70,058 /4,057,586
dtype: object



##**Table 3**

In [0]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
import statsmodels.formula.api as smf
import statsmodels.api as sm
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
utils = importr('utils')
utils.install_packages('msm')
msm = importr('msm')


In [14]:
# Truncate follow-up time
def truncate(t):
    e = d.copy()
    if t>0:
      e.loc[e['wl_time'] > t, 'wl']=0
      e.loc[e['wl_time'] > t, 'wl_time']=t
      e.loc[e['ld_time'] > t, 'livingd']=0
      e.loc[e['ld_time'] > t, 'ld_time']=t
      e.loc[e['dec_time'] > t, 'deceasedt']=0
      e.loc[e['dec_time'] > t, 'dec_time']=t
    return e

# dataset with aggregate sum for poisson model
for event, time in zip(['wl','livingd', 'deceasedt'],['wl_time', 'ld_time', 'dec_time']):
	for cutoff in [36, 60, 120]:
		poissonset = truncate(cutoff).groupby(['for_profit', 'age_cat', 'sex_new', 'race_new'], as_index=False)[['wl', 'wl_time', 'livingd', 'ld_time', 'dec_time', 'deceasedt']].sum()
		poissonset['age_cat'] = pd.Categorical(poissonset['age_cat'], ordered=True, categories=[5, 1, 2, 3, 4, 6])
		poissonset['race_new'] = pd.Categorical(poissonset['race_new'], ordered=True, categories=[1, 2, 3, 4])

		# poisson regression
		poisson_model = smf.glm(event+'~for_profit+sex_new+age_cat+race_new',
								data=poissonset,
								family=sm.families.Poisson(),
								offset=np.log(poissonset[time]/12)).fit(method='newton')

		# extract coefficient
		b0 = poisson_model.params['Intercept']
		b1 = poisson_model.params['for_profit']

		# calculate rate difference and standard error
		effect = np.exp(b0 + b1) - np.exp(b0)
		vcov = poisson_model.cov_params().loc[['Intercept', 'for_profit'], ['Intercept', 'for_profit']]
		se = msm.deltamethod(ro.Formula('~exp(x1+x2)-exp(x1)'), FloatVector([b0, b1]), ro.r.matrix(vcov.values, nrow=2, ncol=2))
		print("%s %g-months for_profit vs. non_profit: %.2f (%.2f, %.2f) " %(event, cutoff, effect*100, (effect-1.96*float(np.array(se)))*100, (effect+1.96*float(np.array(se)))*100))  
    

wl 36-months for_profit vs. non_profit: -1.24 (-1.35, -1.13) 
wl 60-months for_profit vs. non_profit: -0.92 (-1.01, -0.84) 
wl 120-months for_profit vs. non_profit: -0.65 (-0.72, -0.58) 
livingd 36-months for_profit vs. non_profit: -0.33 (-0.37, -0.28) 
livingd 60-months for_profit vs. non_profit: -0.26 (-0.29, -0.22) 
livingd 120-months for_profit vs. non_profit: -0.20 (-0.23, -0.18) 
deceasedt 36-months for_profit vs. non_profit: -0.38 (-0.43, -0.33) 
deceasedt 60-months for_profit vs. non_profit: -0.42 (-0.46, -0.37) 
deceasedt 120-months for_profit vs. non_profit: -0.32 (-0.35, -0.28) 


In [15]:
for event, time in zip(['wl','livingd', 'deceasedt'],['wl_time', 'ld_time', 'dec_time']):
  for cutoff in [36, 60, 120]:
    poissonset = truncate(cutoff).groupby(['chain_class2', 'age_cat', 'sex_new', 'race_new'], as_index=False)[['wl', 'wl_time', 'livingd', 'ld_time', 'dec_time', 'deceasedt']].sum()
    poissonset['chain_class2'] = pd.Categorical(poissonset['chain_class2'], ordered=True, categories=[6, 5, 2, 1, 3, 4])
    poissonset['age_cat'] = pd.Categorical(poissonset['age_cat'], ordered=True, categories=[5, 1, 2, 3, 4, 6])
    poissonset['race_new'] = pd.Categorical(poissonset['race_new'], ordered=True, categories=[1, 2, 3, 4])

    # poisson regression
    poisson_model = smf.glm(event+'~chain_class2+sex_new+age_cat+race_new',
                            data=poissonset,
                            family=sm.families.Poisson(),
                            offset=np.log(poissonset[time]/12)).fit(method='newton')

		# extract coefficient
    for i in [5, 2, 1, 3, 4]:
      b0 = poisson_model.params['Intercept']
      b1 = poisson_model.params['chain_class2[T.'+str(i)+']']
      effect = np.exp(b0 + b1) - np.exp(b0)
      vcov = poisson_model.cov_params().loc[['Intercept', 'chain_class2[T.'+str(i)+']'], ['Intercept', 'chain_class2[T.'+str(i)+']']]
      se = msm.deltamethod(ro.Formula('~exp(x1+x2)-exp(x1)'), FloatVector([b0, b1]), ro.r.matrix(vcov.values, nrow=2, ncol=2))
      print("%s %g-months chain_%g: %.2f (%.2f, %.2f) " %(event, cutoff, i, effect*100, (effect-1.96*float(np.array(se)))*100, (effect+1.96*float(np.array(se)))*100))

wl 36-months chain_5: -0.00 (-0.21, 0.21) 
wl 36-months chain_2: -1.14 (-1.28, -1.00) 
wl 36-months chain_1: -1.22 (-1.37, -1.08) 
wl 36-months chain_3: -1.34 (-1.49, -1.19) 
wl 36-months chain_4: -1.55 (-1.73, -1.37) 
wl 60-months chain_5: -0.08 (-0.25, 0.09) 
wl 60-months chain_2: -0.88 (-0.99, -0.76) 
wl 60-months chain_1: -0.96 (-1.07, -0.85) 
wl 60-months chain_3: -1.01 (-1.13, -0.89) 
wl 60-months chain_4: -1.19 (-1.33, -1.04) 
wl 120-months chain_5: -0.16 (-0.29, -0.02) 
wl 120-months chain_2: -0.65 (-0.75, -0.56) 
wl 120-months chain_1: -0.73 (-0.82, -0.63) 
wl 120-months chain_3: -0.73 (-0.83, -0.62) 
wl 120-months chain_4: -0.89 (-1.01, -0.77) 
livingd 36-months chain_5: 0.16 (0.07, 0.24) 
livingd 36-months chain_2: -0.26 (-0.31, -0.20) 
livingd 36-months chain_1: -0.24 (-0.30, -0.19) 
livingd 36-months chain_3: -0.35 (-0.41, -0.29) 
livingd 36-months chain_4: -0.25 (-0.32, -0.17) 
livingd 60-months chain_5: 0.12 (0.06, 0.19) 
livingd 60-months chain_2: -0.20 (-0.25, -0.16) 


In [16]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain_class2_', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains(exposure)].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


------------------------------ wl ------------------------------

 model_0
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.04             1.01             1.06
chain_class2_2  0.90             0.88             0.91
chain_class2_1  0.85             0.84             0.87
chain_class2_3  0.87             0.86             0.89
chain_class2_4  0.82             0.80             0.84

 model_1
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  1.00             0.97             1.02
chain_class2_2  0.89             0.87             0.90
chain_class2_1  0.87             0.86             0.88
chain_class2_3  0.86             0.85             0.88
chain_class2_4  0.84             0.82             0.86

 model_2
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  0.99             0.97             1.02
chain_class2_2  0.87             0.86             0.88
chain_class2_1  0.88             0.86             0.89
chain_class2_3  0.86     


##**Figure 2**

In [17]:
d['FIRST_SE'] = pd.to_datetime(d['FIRST_SE'], format='%m/%d/%Y')
fig2 = pd.DataFrame()

for event, time in zip(['wl', 'livingd', 'deceasedt'], ['wl_time', 'ld_time', 'dec_time']):
  print('-'*40, event, '-'*40)
  for exposure in ['chain_class2', 'for_profit']:
    print('\n', exposure)
    fig2 = pd.DataFrame()
    for year in range(2001, 2016, 2):
        valid = d[(d['FIRST_SE']>=str(year)+'/01/01') & (d['FIRST_SE']<=str(year+1)+'/12/31')]
        validtime = (pd.to_datetime(str(year+1)+'/12/31') - valid['FIRST_SE']).apply(lambda x: x.days/30.4375)
        valid.loc[valid[time]>validtime, event] = 0
        valid.loc[valid[time]>validtime, time] = validtime
        num = valid.groupby(exposure)[event].sum()
        pyr = valid.groupby(exposure)[time].sum()/12
        rate = (num/pyr*100).rename(str(year)+'-'+"{0:0>2}".format(year+1-2000))
        fig2 = pd.concat([fig2, rate], 1)
    print(fig2.applymap('{:.2f}'.format))

---------------------------------------- wl ----------------------------------------

 chain_class2
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
1    7.45    7.52    7.97    8.12    8.39    8.19    7.75    6.36
2    7.86    7.89    8.77    8.16    8.30    8.30    8.50    6.83
3    7.27    7.78    7.95    7.49    8.67    7.92    7.67    6.54
4    6.79    7.16    7.53    7.96    7.31    7.23    7.22    6.33
5    8.87   10.24   11.25   11.95   11.06   11.01   10.54    8.86
6    9.64    9.75   10.85   10.12   10.43   10.86    9.84    8.03

 for_profit
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
0    9.37    9.92   10.99   10.80   10.66   10.92   10.10    8.33
1    7.54    7.68    8.24    8.01    8.32    8.11    7.98    6.57
---------------------------------------- livingd ----------------------------------------

 chain_class2
  2001-02 2003-04 2005-06 2007-08 2009-10 2011-12 2013-14 2015-16
1    1.60    1.59    1.32    1.17    0.91    0.84    0.7

##Supplemental Table 2

In [18]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for i in [6,5,2,1,3,4]:
    print('\n chain_class2={}'.format(i))
    crude = '|'.join(['change_status', status, time])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn'
    if i in [5,6]:
      tmp = PH_data[PH_data.chain_class2==i].filter(regex=model3).drop(['change_status_3'], 1)
    else:
      tmp = PH_data[PH_data.chain_class2==i].filter(regex=model3).drop(['change_status_2'], 1)
    cph.fit(tmp,duration_col=time, event_col=status, step_size=0.5)
    print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains('change_status_')].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains('change_status_')])], 1), 2))

------------------------------ wl ------------------------------

 chain_class2=6
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.72             0.69             0.75
change_status_2  0.61             0.58             0.64

 chain_class2=5
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.77             0.72             0.83
change_status_2  0.60             0.57             0.64

 chain_class2=2
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.62             0.61             0.63
change_status_3  0.81             0.77             0.84

 chain_class2=1
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.62             0.61             0.63
change_status_3  0.79             0.76             0.83

 chain_class2=3
                   HR  95% lower-bound  95% upper-bound
change_status_1  0.61             0.59             0.62
change_status_3  0.80             0.76             0.85

 chain_cl


##**Supplemental Table 3 (ideal cohort)**

In [19]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['chain_class2_', 'for_profit']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                  'copd_new|smoke_new|cancer_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|network_us_region_dfr|NEAR_DIST|rucc_rural' #PATTXOP_MEDUNFITn'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model'+str(i))
      cph.fit(PH_data_ideal.filter(regex=model), duration_col=time, event_col=status, step_size=0.5)
      print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains(exposure)].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


------------------------------ wl ------------------------------

 model0
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  0.97             0.95             1.00
chain_class2_2  0.85             0.84             0.87
chain_class2_1  0.83             0.81             0.85
chain_class2_3  0.84             0.83             0.86
chain_class2_4  0.84             0.82             0.86

 model1
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  0.96             0.93             0.98
chain_class2_2  0.87             0.85             0.88
chain_class2_1  0.86             0.84             0.88
chain_class2_3  0.86             0.84             0.87
chain_class2_4  0.85             0.83             0.87

 model2
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  0.96             0.93             0.99
chain_class2_2  0.87             0.86             0.89
chain_class2_1  0.88             0.86             0.90
chain_class2_3  0.87        


##**Supplemental Table 4**

In [20]:
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for r, region in enumerate(['Northeast', 'South', 'Midwest', 'West']):
    print('\n', region)
    for exposure in ['chain_class2_', 'for_profit']:
      crude = '|'.join([exposure, time, status])
      model1 = crude + '|sex_new|age_cat|race_new'
      model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                        'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
      model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn' #network_us_region_dfr'
      cph.fit(PH_data[d['network_us_region_dfr']==r].filter(regex=model3), duration_col=time, event_col=status, step_size=0.3)
      print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains(exposure)].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))

------------------------------ wl ------------------------------

 Northeast
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  0.85             0.81             0.89
chain_class2_2  0.91             0.87             0.95
chain_class2_1  0.86             0.83             0.90
chain_class2_3  0.86             0.81             0.90
chain_class2_4  0.85             0.80             0.89
              HR  95% lower-bound  95% upper-bound
for_profit  0.98             0.95              1.0

 South
                  HR  95% lower-bound  95% upper-bound
chain_class2_5  0.91             0.86             0.96
chain_class2_2  0.86             0.84             0.89
chain_class2_1  0.92             0.89             0.94
chain_class2_3  0.93             0.90             0.96
chain_class2_4  0.85             0.82             0.88
              HR  95% lower-bound  95% upper-bound
for_profit  0.92             0.89             0.94

 Midwest
                  HR  95% lower-bound  95


##Supplemental Table 5 & 6

In [21]:
cph = CoxPHFitter()
for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  print('-'*30, status,'-'*30)
  for exposure in ['profit_txc', 'profit_hosp']:
    crude = '|'.join([exposure, time, status])
    model1 = crude + '|sex_new|age_cat|race_new'
    model2 = model1 + '|dialysis_mod1|esrd_cause|bmi_35|ashd_new|other_cardiac|hypertension|diabetes|'\
                      'copd_new|smoke_new|cancer_new|chf|cva_new|pvasc_new'#chf|cva_new|pvasc_new'
    model3 = model2 + '|insurance_esrd|NEAR_DIST|rucc_rural|PATTXOP_MEDUNFITn|network_us_region_dfr' #network_us_region_dfr'
    for i, model in enumerate([crude, model1, model2, model3]):
      print('\n', 'model_'+str(i))
      cph.fit(PH_data.filter(regex=model), duration_col=time, event_col=status, step_size=0.2)
      print(round(pd.concat([cph.hazard_ratios_[cph.hazard_ratios_.index.str.contains(exposure)].rename('HR'), exp(cph.confidence_intervals_[cph.confidence_intervals_.index.str.contains(exposure)])], 1), 2))


------------------------------ wl ------------------------------

 model_0
                HR  95% lower-bound  95% upper-bound
profit_txc_1  0.59             0.57             0.62
profit_txc_2  0.92             0.79             1.07
profit_txc_3  0.52             0.50             0.55

 model_1
                HR  95% lower-bound  95% upper-bound
profit_txc_1  0.72             0.69             0.76
profit_txc_2  1.03             0.88             1.19
profit_txc_3  0.64             0.62             0.67

 model_2
                HR  95% lower-bound  95% upper-bound
profit_txc_1  0.80             0.77             0.84
profit_txc_2  1.01             0.87             1.18
profit_txc_3  0.71             0.68             0.74

 model_3
                HR  95% lower-bound  95% upper-bound
profit_txc_1  0.87             0.83             0.91
profit_txc_2  1.04             0.89             1.20
profit_txc_3  0.76             0.73             0.80

 model_0
                 HR  95% lower-bound 

## Additional: Cumulative incidence difference

In [23]:
cif = d.copy()
cif.loc[cif['death_wl']==1, 'wl']=2
cif.loc[cif['death_ld']==1, 'livingd']=2
cif.loc[cif['death_dec']==1, 'deceasedt']=2
d_p = cif[cif.for_profit==1]
d_np = cif[cif.for_profit==0]

for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  ajf_p = AalenJohansenFitter(calculate_variance=False).fit(d_p[time]/12, d_p[status], 1)
  ajf_np = AalenJohansenFitter(calculate_variance=False).fit(d_np[time]/12, d_np[status], 1)
  for t in [3, 5, 10]:
    cif_p = ajf_p.cumulative_density_.loc[slice(t)].tail(1).values
    cif_np = ajf_np.cumulative_density_.loc[slice(t)].tail(1).values
    cif_diff = cif_p - cif_np
    se = np.sqrt(cif_p * (1-cif_p) / len(d_p) +cif_np * (1-cif_np) / len(d_np))
    print('%s %g-year profit vs. non-profit: %.1f%% (%.1f%%, %.1f%%)' 
          %(status, t, cif_diff*100, (cif_diff -1.96 *se)*100, (cif_diff+1.96*se )*100))

# for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
#   ajf_p = AalenJohansenFitter(calculate_variance=True).fit(d_p[time]/12, d_p[status], 1)
#   ajf_np = AalenJohansenFitter(calculate_variance=True).fit(d_np[time]/12, d_np[status], 1)
  
#   cif_p = ajf_p.cumulative_density_.loc[slice(10)].tail(1).join(ajf_p.confidence_interval_.loc[slice(10)].tail(1))
#   cif_np = ajf_np.cumulative_density_.loc[slice(10)].tail(1).join(ajf_np.confidence_interval_.loc[slice(10)].tail(1))
#   print(cif_p, cif_np)

wl 3-year profit vs. non-profit: -2.3% (-2.4%, -2.1%)
wl 5-year profit vs. non-profit: -2.3% (-2.5%, -2.1%)
wl 10-year profit vs. non-profit: -2.2% (-2.4%, -2.0%)
livingd 3-year profit vs. non-profit: -0.8% (-0.9%, -0.8%)
livingd 5-year profit vs. non-profit: -0.9% (-1.0%, -0.8%)
livingd 10-year profit vs. non-profit: -0.9% (-1.0%, -0.8%)
deceasedt 3-year profit vs. non-profit: -0.8% (-0.9%, -0.8%)
deceasedt 5-year profit vs. non-profit: -1.4% (-1.5%, -1.2%)
deceasedt 10-year profit vs. non-profit: -1.5% (-1.6%, -1.4%)


In [24]:
d_c1 = cif[cif.chain_class2==1]
d_c2 = cif[cif.chain_class2==2]
d_c3 = cif[cif.chain_class2==3]
d_c4 = cif[cif.chain_class2==4]
d_c5 = cif[cif.chain_class2==5]
d_c6 = cif[cif.chain_class2==6]


for time, status in zip(['wl_time', 'ld_time', 'dec_time'], ['wl', 'livingd', 'deceasedt']):
  ajf_c1 = AalenJohansenFitter(calculate_variance=False).fit(d_c1[time]/12, d_c1[status], 1)
  ajf_c2 = AalenJohansenFitter(calculate_variance=False).fit(d_c2[time]/12, d_c2[status], 1)
  ajf_c3 = AalenJohansenFitter(calculate_variance=False).fit(d_c3[time]/12, d_c3[status], 1)
  ajf_c4 = AalenJohansenFitter(calculate_variance=False).fit(d_c4[time]/12, d_c4[status], 1)
  ajf_c5 = AalenJohansenFitter(calculate_variance=False).fit(d_c5[time]/12, d_c5[status], 1)
  ajf_c6 = AalenJohansenFitter(calculate_variance=False).fit(d_c6[time]/12, d_c6[status], 1)
  for c, d_len, n in zip([ajf_c5, ajf_c2, ajf_c1, ajf_c3, ajf_c4], [d_c5, d_c2, d_c1, d_c3, d_c4], [5, 2, 1, 3, 4]):
    for t in [3, 5, 10]:
      cif_c6 = ajf_c6.cumulative_density_.loc[slice(t)].tail(1).values
      cif_c = c.cumulative_density_.loc[slice(t)].tail(1).values
      cif_diff = cif_c - cif_c6
      se = np.sqrt(cif_c6 * (1-cif_c6) / len(d_c6) +cif_c * (1-cif_c) / len(d_len))
      print('%s %g-year chain_class %g vs. 6: %.1f%% (%.1f%%, %.1f%%)' 
            %(status, t, n, cif_diff*100, (cif_diff -1.96 *se)*100, (cif_diff+1.96*se )*100))

wl 3-year chain_class 5 vs. 6: 0.5% (0.2%, 0.9%)
wl 5-year chain_class 5 vs. 6: 0.6% (0.2%, 0.9%)
wl 10-year chain_class 5 vs. 6: 0.8% (0.4%, 1.2%)
wl 3-year chain_class 2 vs. 6: -1.6% (-1.8%, -1.4%)
wl 5-year chain_class 2 vs. 6: -1.5% (-1.8%, -1.3%)
wl 10-year chain_class 2 vs. 6: -1.3% (-1.5%, -1.0%)
wl 3-year chain_class 1 vs. 6: -2.3% (-2.6%, -2.1%)
wl 5-year chain_class 1 vs. 6: -2.5% (-2.7%, -2.2%)
wl 10-year chain_class 1 vs. 6: -2.3% (-2.6%, -2.1%)
wl 3-year chain_class 3 vs. 6: -2.0% (-2.3%, -1.8%)
wl 5-year chain_class 3 vs. 6: -2.0% (-2.2%, -1.7%)
wl 10-year chain_class 3 vs. 6: -1.8% (-2.1%, -1.6%)
wl 3-year chain_class 4 vs. 6: -3.1% (-3.4%, -2.8%)
wl 5-year chain_class 4 vs. 6: -3.1% (-3.4%, -2.8%)
wl 10-year chain_class 4 vs. 6: -3.1% (-3.4%, -2.7%)
livingd 3-year chain_class 5 vs. 6: 0.5% (0.3%, 0.6%)
livingd 5-year chain_class 5 vs. 6: 0.6% (0.4%, 0.7%)
livingd 10-year chain_class 5 vs. 6: 0.7% (0.5%, 0.8%)
livingd 3-year chain_class 2 vs. 6: -0.6% (-0.7%, -0.5%)
livi